### Read Public Groups/Pages classify groups by State
### Topic Modeling 

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from collections import Counter

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

#TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer

# spacy for lemmatization
import spacy

#nltk
from nltk.corpus import stopwords

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
def clean_address(row):
    name =  row.lower().split('/')
    if  row.lower().startswith('https') or  row.lower().startswith('http'):       
        return name[3]
    elif row.lower().startswith('www'):
        return name[1] 
    elif row.lower().startswith('facebook'):
        return name[1]
    else:
        print("Not found", row)
        return None

## 1. Reads  Info
Reads the information from all the posts from the studied period

In [3]:

path = "cscwDocs/parties/FBPagesGroupsData/FBPagesGroupsCities/"
posts1 = pd.read_csv(path+"LatinosEstadosPagesAug182020-Nov032020.csv")
#We rename the colum Page Name to be able to concat with Group Posts
posts1.rename(columns={'Page Name':'Group Name'}, inplace=True)
posts2 = pd.read_csv(path+"LatinosEstadosGroups-Aug182020-Aug252020.csv")
posts3 = pd.read_csv(path+"LatinosEstadosGroups-Aug262020-Aug312020.csv")
posts4 = pd.read_csv(path+"LatinosEstadosGroups-Sep012020-Sep042020.csv")
posts5 = pd.read_csv(path+"LatinosEstadosGroups-Sep052020-Sep082020.csv")
posts6 = pd.read_csv(path+"LatinosEstadosGroups-Sep092020-Sep122020.csv")
posts7 = pd.read_csv(path+"LatinosEstadosGroups-Sep132020-Sep162020.csv")
posts8 = pd.read_csv(path+"LatinosEstadosGroups-Sep172020-Sep202020.csv")
posts9 = pd.read_csv(path+"LatinosEstadosGroups-Sep212020-Sep242020.csv")
posts10 = pd.read_csv(path+"LatinosEstadosGroups-Sep252020-Sep282020.csv")
posts11 = pd.read_csv(path+"LatinosEstadosGroups-Sep292020-Oct022020.csv")
posts12 = pd.read_csv(path+"LatinosEstadosGroups-Oct032020-Oct062020.csv")
posts13 = pd.read_csv(path+"LatinosEstadosGroups-Oct072020-Oct102020.csv")
posts14 = pd.read_csv(path+"LatinosEstadosGroups-Oct112020-Oct142020.csv")
posts15 = pd.read_csv(path+"LatinosEstadosGroups-Oct152020-Oct172020.csv")
posts16 = pd.read_csv(path+"LatinosEstadosGroups-Oct182020--Oct212020.csv")
posts17 = pd.read_csv(path+"LatinosEstadosGroups-Oct222020-Oct252020.csv")
posts18 = pd.read_csv(path+"LatinosEstadosGroups-Oct262020-Oct292020.csv")
posts19 = pd.read_csv(path+"LatinosEstadosGroups-October302020-Nov032020.csv")
posts = pd.concat([posts1,posts2,posts3,posts4,posts5,posts6,posts7,posts8,posts9,
                   posts10,posts11,posts12,posts13,posts14,posts15,posts16,posts17,posts18,posts19])
posts.shape

/Users/claudia/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/claudia/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(1064972, 31)

In [4]:
posts['URL']

0        https://www.facebook.com/massbhr/posts/1707320...
1        https://www.facebook.com/CampuzanoDallas/posts...
2        https://www.facebook.com/bluecrossblueshieldof...
3        https://www.facebook.com/CampuzanoDallas/posts...
4        https://www.facebook.com/IHCHOUSTON/posts/6120...
                               ...                        
65350    https://www.facebook.com/groups/16251973944111...
65351    https://www.facebook.com/groups/16251973944111...
65352    https://www.facebook.com/groups/16251973944111...
65353    https://www.facebook.com/groups/16251973944111...
65354    https://www.facebook.com/groups/16251973944111...
Name: URL, Length: 1064972, dtype: object

## 9. Filter the most relevant columns
### Add column to concat Message, Description, Link Text

In [5]:
concat_message = posts['Message'].fillna('') + (' ' + posts['Description']).fillna('')+ (' ' + posts['Link Text']).fillna('') + (' ' + posts['Link']).fillna('')
posts['MessageDescr'] = concat_message

concat_just_message = posts['Message'].fillna('') + (' ' + posts['Description']).fillna('')+ (' ' + posts['Link Text']).fillna('')
posts['MessageOnly'] = concat_just_message

In [6]:
posts.columns

Index(['Group Name', 'User Name', 'Facebook Id', 'Likes at Posting', 'Created',
       'Type', 'Likes', 'Comments', 'Shares', 'Love', 'Wow', 'Haha', 'Sad',
       'Angry', 'Care', 'Video Share Status', 'Post Views', 'Total Views',
       'Total Views For All Crossposts', 'Video Length', 'URL', 'Message',
       'Link', 'Final Link', 'Image Text', 'Link Text', 'Description',
       'Sponsor Id', 'Sponsor Name', 'Total Interactions',
       'Overperforming Score', 'MessageDescr', 'MessageOnly'],
      dtype='object')

In [7]:
posts[posts['Link Text'].str.lower().str.contains("í", na = False)].head(1)

,Group Name,User Name,Facebook Id,Likes at Posting,Created,Type,Likes,Comments,Shares,Love,...,Final Link,Image Text,Link Text,Description,Sponsor Id,Sponsor Name,Total Interactions,Overperforming Score,MessageDescr,MessageOnly
4,Institute of Hispanic Culture of Houston,IHCHOUSTON,1715255265365568,5778.0,2020-08-20 18:00:48 EDT,Native Video,1032,56,106,28,...,NaN,NaN,SPOTLIGHTS VENEZUELA Capítulo 2 con el percusi...,NaN,NaN,NaN,"1,227",40.9,Conversaremos con este famoso percusionista Ve...,Conversaremos con este famoso percusionista Ve...


In [8]:
#!/usr/bin/env python
import re

text = u'This is a smiley face \U0001f602'
print(text) # with emoji

def deEmojify(text):
    
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                          
                           "]+", flags = re.UNICODE)
    
    
    return regrex_pattern.sub(r'',text)

print(deEmojify(text))

def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

This is a smiley face 😂
This is a smiley face 


In [9]:

posts['Group Name']=posts['Group Name'].apply(deEmojify)
posts['Group Name']=posts['Group Name'].apply(lambda i: i.replace('"', ''))
posts['Group Name']=posts['Group Name'].apply(lambda i: i.replace(',', ''))
posts['Facebook Id'] = posts['Facebook Id'].astype(int)

## Read FB Group Names & State


In [10]:
fb_groups = pd.read_csv("cscwDocs/parties/FBGroupsPolitical_cities.csv")
fb_groups['Facebook Id'] = fb_groups['Page or Account URL'].str.strip("https://www.facebook.com/groups/")
fb_groups

,name,Page or Account URL,List,State,state,Facebook Id
0,Hispanos unidos en New York (H.U.N.Y),https://www.facebook.com/groups/793416207356486,Latinos+EstadosGroups,New York,New York,793416207356486
1,Hispanos en Columbus Ohio,https://www.facebook.com/groups/1450333408570918,Latinos+EstadosGroups,Ohio,Ohio,1450333408570918
2,"Cristianos Hispanos en Dallas, Texas ( Metroplex)",https://www.facebook.com/groups/600361880064744,Latinos+EstadosGroups,Texas,Texas,600361880064744
3,Hispanos en Oklahoma,https://www.facebook.com/groups/1160877743977648,Latinos+EstadosGroups,Oklahoma,Oklahoma,1160877743977648
4,Hispanos unidos en el gabacho (Tennessee),https://www.facebook.com/groups/2663330647279121,Latinos+EstadosGroups,Tennessee,Tennessee,2663330647279121
...,...,...,...,...,...,...
492,Latinos en SWFL,https://www.facebook.com/groups/125398434802756,Latinos+EstadosGroups,Florida,Florida,125398434802756
493,Latinos en Fayetteville NC,https://www.facebook.com/groups/330153457746158,Latinos+EstadosGroups,North Carolina,North Carolina,330153457746158
494,LATINOS EN UTAH,https://www.facebook.com/groups/147690845007,Latinos+EstadosGroups,Utah,Utah,147690845007
495,Latinos en los Angeles California compra y venta,https://www.facebook.com/groups/166308334272338,Latinos+EstadosGroups,California,California,166308334272338


### 4. US states

In [11]:
us_states = pd.DataFrame({'State':['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','District of Columbia','Florida',
             'Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts',
             'Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico',
             'New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina',
             'South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']})

##  Count posts in general by state

In [12]:
from functools import reduce
data_frames = [fb_groups, posts]
fb_groups['Facebook Id'] = fb_groups['Facebook Id'].astype(int)
df_posts_state  = reduce(lambda  left,right: pd.merge(left,right,on=['Facebook Id'],
                                            how='outer'), data_frames)
df_count = df_posts_state.groupby(['State']).agg({'Facebook Id':'count'}).reset_index() 

df_count.columns = ['State','Posts_Counts']
df_count.sort_values(by=['State'], ascending=True,inplace=True)
df_count.fillna(0, inplace=True)
df_count['Posts_Counts'] = df_count['Posts_Counts'].astype(int)
df_count.head(10)

,State,Posts_Counts
0,Arizona,4140
1,Arkansas,44
2,California,61055
3,Colorado,1645
4,Connecticut,9245
5,Florida,303098
6,Georgia,23082
7,Illinois,16804
8,Indiana,103
9,Iowa,675


In [13]:
#us_states['State'] = us_states['State']
data_frames = [df_count, us_states]
missing_states  = reduce(lambda  left,right: pd.merge(left,right,on=['State'],
                                            how='outer'), data_frames)
missing_states.fillna(0, inplace=True)
missing_states['Posts_Counts'] = missing_states['Posts_Counts'].astype(int)
missing_states

,State,Posts_Counts
0,Arizona,4140
1,Arkansas,44
2,California,61055
3,Colorado,1645
4,Connecticut,9245
5,Florida,303098
6,Georgia,23082
7,Illinois,16804
8,Indiana,103
9,Iowa,675


In [14]:
df_count.to_csv('cscwDocs/FB_groups_States/FB_groups_posts_by_state2.csv',index=False,na_rep=0)

In [15]:
df_posts_state.columns

Index(['name', 'Page or Account URL', 'List', 'State', 'state', 'Facebook Id',
       'Group Name', 'User Name', 'Likes at Posting', 'Created', 'Type',
       'Likes', 'Comments', 'Shares', 'Love', 'Wow', 'Haha', 'Sad', 'Angry',
       'Care', 'Video Share Status', 'Post Views', 'Total Views',
       'Total Views For All Crossposts', 'Video Length', 'URL', 'Message',
       'Link', 'Final Link', 'Image Text', 'Link Text', 'Description',
       'Sponsor Id', 'Sponsor Name', 'Total Interactions',
       'Overperforming Score', 'MessageDescr', 'MessageOnly'],
      dtype='object')

## 3. Which groups are publishing the majority of the content

In [16]:
group = posts.groupby(['Group Name'])
final_groups = group.agg({'Facebook Id':'count'}).sort_values(by='Facebook Id',ascending=False)
final_groups

,Facebook Id
Group Name,
LATINOS EN NEW YORK,73716
Venezolanos en Miami,29886
Latinos en New Jersey y areas cercanas,20811
Cubanos en miamifl,16746
Latinos en Minnesota,16011
...,...
New York State Black Puerto Rican Hispanic and Asian Legislative Caucus,1
Latino San Francisco Bay Area,1
North Carolina Congress of Latino Organizations,1


In [17]:
final_groups.to_csv("cscwDocs/FB_groups_States/Posts_Per_group2.csv")

### 6.Filter Posts per each political party
How much each candidate talked about all the topics that Latinx cared about

In [18]:
#Topics
economy = 'economy|economía'
health_care = 'health care|salud'
coronavirus = 'coronavirus|covid'
crime = 'crime'
supreme_court = 'supreme court|suprema corte'
climate_change = 'climate change|cambio climático'
immigration = 'immigration|migration|inmigración|migración'
gun_policy = 'gun policy|gun policies|política de armas|guns rights|NRA|armas|arma'
foreign_policy ='foreign policy|política exterior|foreign policies'
abortion = 'aborto|abortion'
racism = 'racial inequality|etnic inqueality|racism|racismo|inequidad racial|inequidad étnica'

all_topics = economy + '|' + health_care + '|' + coronavirus  + '|' + crime  + '|' + supreme_court + '|' + climate_change  + '|' + immigration  + '|' + gun_policy + '|' + foreign_policy + abortion + racism


republicans = "donald trump|trump|mike pence|nikki haley|john mccain|susan collins|rand paul|chris christie|\
jeff flake|roger stone|george bush|george w. bush|herbert hoover|charles curtis|sarah palin|bill weld|\
rocky de la fuente|joe walsh|mark sanford|herman cain|brad parscale|melania trump|richard nixon|jared kushner|\
ivanka trump|lindsey graham|newt gingrich|rudy giuliani|william barr|mitch mcconnell|gop|republican|\
republicans|republicano|republicanos"


#democrats
democrats = 'joe biden|biden|hillary clinton|kamala harris|michelle obama|bernie sanders|andrew cuomo|tom pérez|\
harry reid|bill clinton|barack obama|adlai stevenson ii|pete buttigieg|michael bennet|deval patrick|andrew yang|\
amy klobuchar|tom steyer|michael bloomberg|elizabeth warren|tulsi gabbard|geraldine ferraro|tony evers|\
nancy pelosi|josh shapiro|democrat|democrats|democratas|democrata'


general_politics = "kanye west|jo jorgensen|spike cohen|howie hawkins|angela walker|don blankenship|gloria la riva|\
brian t. carroll|brock pierce|phil collins|bill hammons|jerome segal|blake huber|mark charles|dario hunter|\
joseph kishore|alyson kennedy|jade simmons|joe mchugh|kyle kopitke|amy coney barrett|ruth bader ginsburg|\
george floyd|oprah winfrey|breonna taylor|geng shuang|philip goldberg|jho low|chris krebs|chris wallace|\
christopher a. wray|louis dejoy|merrick garland|steve scully|susan page|kristen welker|urszula gacek|\
michael georg link|william h. frey|ann selzer|samuel alito|tucker carlson|cynthia stephens|sidney powell|\
shelby pierson|joseph maguire"



posts_democrats = df_posts_state.copy()
posts_democrats = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(democrats   ,na = False)]
print('democrats', posts_democrats.shape)

posts_republicans = df_posts_state.copy()
posts_republicans = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(republicans   ,na = False)]
print('republicans', posts_republicans.shape)

posts_general_politics = df_posts_state.copy()
posts_general_politics = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(general_politics   ,na = False)]
print('general politics', posts_general_politics.shape)


posts_none = df_posts_state.copy()
posts_none = df_posts_state[~df_posts_state['MessageDescr'].str.lower().str.contains(general_politics + '|' + democrats + '|' +republicans ,na = False)]
print('none',posts_none['MessageDescr'].shape)

df_posts_state['Link Text'].isna().sum()

democrats (10855, 38)
republicans (20141, 38)
general politics (886, 38)
none (1039231,)


426659

In [19]:
posts_democrats['MessageDescr'][3:6]

382     Una noche de ataques contra el Pdte. Trump, l...
396     Home Nacionales INFORME: Se inició la inédita...
750    Agentes federales abatan a un hombre sospechos...
Name: MessageDescr, dtype: object

In [20]:
posts_democrats.columns

Index(['name', 'Page or Account URL', 'List', 'State', 'state', 'Facebook Id',
       'Group Name', 'User Name', 'Likes at Posting', 'Created', 'Type',
       'Likes', 'Comments', 'Shares', 'Love', 'Wow', 'Haha', 'Sad', 'Angry',
       'Care', 'Video Share Status', 'Post Views', 'Total Views',
       'Total Views For All Crossposts', 'Video Length', 'URL', 'Message',
       'Link', 'Final Link', 'Image Text', 'Link Text', 'Description',
       'Sponsor Id', 'Sponsor Name', 'Total Interactions',
       'Overperforming Score', 'MessageDescr', 'MessageOnly'],
      dtype='object')

In [21]:
df_dem = posts_democrats['State'].value_counts().reset_index()
df_dem.columns = ['State','Count']

In [22]:
df_rep = posts_republicans['State'].value_counts().reset_index()
df_rep.columns = ['State','Count']

In [23]:
df_gen_pol = posts_general_politics['State'].value_counts().reset_index()
df_gen_pol.columns = ['State','Count']

In [24]:
df_gen_info = posts_none['State'].value_counts().reset_index()
df_gen_info.columns = ['State','Count']

In [25]:
from functools import reduce
data_frames = [df_dem, df_rep,df_gen_pol,df_gen_info]
df_all = reduce(lambda  left,right: pd.merge(left,right,on=['State'],
                                            how='outer'), data_frames)
df_all.columns = ['State','Posts_Dem','Posts_Rep', 'Posts_Gen_Pol','Post_Gen_Info']
df_all.sort_values(by=['State'], ascending=True,inplace=True)
df_all.fillna(0, inplace=True)
df_all['Posts_Dem'] = df_all['Posts_Dem'].astype(int)
df_all['Posts_Rep'] = df_all['Posts_Rep'].astype(int)
df_all['Posts_Gen_Pol'] = df_all['Posts_Gen_Pol'].astype(int)
df_all['Post_Gen_Info'] = df_all['Post_Gen_Info'].astype(int)
df_all

,State,Posts_Dem,Posts_Rep,Posts_Gen_Pol,Post_Gen_Info
22,Arizona,13,6,0,4125
32,Arkansas,1,2,1,42
1,California,1904,3825,152,56200
27,Colorado,8,5,2,1631
16,Connecticut,48,109,5,9118
0,Florida,3496,6354,219,294946
15,Georgia,57,111,0,22950
11,Illinois,103,225,5,16543
30,Indiana,2,7,0,96
28,Iowa,4,11,2,661


In [26]:
dummy = posts_democrats[posts_democrats['MessageDescr'].str.contains("gop")]
pd.set_option('display.max_colwidth', 400)
dummy['MessageDescr']

58578       Based on electoral history, Democratic challenger Eduardo Castañeda Diaz is facing a steep hill in his attempt to unseat Republican incumbent Tom Dent in the state’s 13th Legislative District. 13th District: Democratic challenger faces uphill fight against GOP incumbent https://www.yakimaherald.com/news/elections/13th-district-democratic-challenger-faces-uphill-fight-against-gop-incumbent/art...
156835     Muchos pero muchos republicanos cambiándose a Democratas. La verdad es que el puerco de Trump esta jodiendo a los Estados Unidos Where is the responsible governance, the individual liberty, equal justice and opportunity for all? I ask my Republican friends: Is this your party? Former New Hampshire GOP chair: My fellow Republicans, Trump does not deserve your loyalty https://www.usatoday.com/st...
165242      46. Brussels is home to the North Atlantic Treaty Organization, one of the US's strongest and most important military alliances. \n\nIt is also important to note that

## Topic Modeling All Posts

## Tokenize words and Clean-up text

In [27]:
content = posts_none.copy()
content['MessageDescr']

0                                                                                                                                                                                                                                                                                                                           Al tiro raza https://www.facebook.com/groups/ENVIANOSTUPROMOGRATIS/permalink/3486515051379908/
1                                                                                                                                                                    Buen dia gente bella 🌹\nLes recuerdo que hoy jueves y viernes pueden hacer sus pedidos de estos ricos productos..\nSe entregan martes..\nsolo por pedido Magda https://www.facebook.com/photo.php?fbid=2011830902280239&set=gm.825505328218205&type=3
2                                                                                                                                                                                 

In [28]:
content['cleanMessage'] = content['MessageDescr'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
content['cleanMessage'].shape

(1039231,)

In [29]:
# Convert to list
data = content.cleanMessage.tolist()

In [30]:
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]


In [31]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:2])

[['al', 'tiro', 'raza'], ['buen', 'dia', 'gente', 'bella', 'les', 'recuerdo', 'que', 'hoy', 'jueves', 'viernes', 'pueden', 'hacer', 'sus', 'pedidos', 'de', 'estos', 'ricos', 'productos', 'se', 'entregan', 'martes', 'solo', 'por', 'pedido', 'magda']]


#### Creating Bigram and Trigram Models

In [32]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['al', 'tiro', 'raza']


## Remove Stopwords, Make Bigrams and Lemmatize

In [33]:
stop_words = stopwords.words('spanish')
stop_words.extend(stopwords.words('english'))
stop_words.extend(['su','sus','al','de','en','el','tal','con','mas','si','dos','pm','ser','ex','could','like','would','get','many','much','\ufeff1'])

In [34]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [35]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#!python3 -m spacy download es_core_news_sm
nlp = spacy.load('es_core_news_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [36]:
print(data_words_nostops[1:2])

[['buen', 'dia', 'gente', 'bella', 'recuerdo', 'hoy', 'jueves', 'viernes', 'pueden', 'hacer', 'pedidos', 'ricos', 'productos', 'entregan', 'martes', 'solo', 'pedido', 'magda']]


## Bag of Words

In [37]:
def combine_words(word1,word2,dictionary):

    if word1 in dictionary and word2 in dictionary:
        concat_name = word1+" / "+word2
        dictionary[concat_name] = dictionary[word1] + dictionary[word2]
        dictionary.pop(word1, None)
        dictionary.pop(word2, None)
        result = dictionary.pop(word1, None)
        result = dictionary.pop(word2, None)
    return dictionary

In [38]:
flat_list_words = [item for sublist in data_lemmatized for item in sublist]
count_words = Counter(flat_list_words)

In [39]:
final_dict = combine_words('donald','trump',count_words)
final_dict = combine_words('joe','biden',count_words)
final_dict = combine_words('kamala','harris',count_words)
final_dict = combine_words('american','america',count_words)
final_dict = combine_words('estados','unidos',count_words)
final_dict = combine_words('voters','vote',count_words)
final_dict = combine_words('mexico','mexican',count_words)
final_dict = combine_words('casa','blanca',count_words)
final_dict = combine_words('venezuela','venezuelans',count_words)
final_dict = combine_words('latino','latinos',count_words)
final_dict = combine_words('american / america','americans',count_words)
final_dict = combine_words('voters / vote','voting',count_words)
final_dict = combine_words('puerto','rico',count_words)
final_dict = combine_words('communism','communist',count_words)
final_dict = combine_words('ee','uu',count_words)
final_dict = combine_words('united','states',count_words)
final_dict = combine_words('president','presidente',count_words)
final_dict = combine_words('democrat','democrats',count_words)
final_dict = combine_words('informacion','informar',count_words)
final_dict = combine_words('covid','coronavirus',count_words)
final_dict = combine_words('democrat / democrats','democratas',count_words)

In [40]:
final_dict.most_common(1000)

[('post', 84009),
 ('hacer', 81376),
 ('solo', 79788),
 ('informacion / informar', 69180),
 ('casar', 67533),
 ('com', 66948),
 ('pagar', 64384),
 ('trabajar', 63613),
 ('hoy', 63564),
 ('photos', 57765),
 ('nuevo', 49098),
 ('querer', 48223),
 ('ayudar', 47314),
 ('personar', 45120),
 ('necesitar', 44168),
 ('mejor', 43886),
 ('servicio', 43723),
 ('bueno', 42427),
 ('dia', 41152),
 ('musicar', 40826),
 ('llamar', 39715),
 ('ano', 39521),
 ('usar', 38391),
 ('derecho', 37608),
 ('whatsapp', 36434),
 ('buscar', 35700),
 ('vivir', 35029),
 ('compartir', 34588),
 ('hola', 34059),
 ('comprar', 32711),
 ('internet', 32123),
 ('vida', 31804),
 ('share', 31802),
 ('mes', 31368),
 ('dias', 31281),
 ('seguro', 30803),
 ('www', 29939),
 ('venta', 29766),
 ('mensaje', 29701),
 ('ver', 29620),
 ('dejar', 29285),
 ('esperar', 28999),
 ('producto', 27500),
 ('cable', 27357),
 ('familia', 27219),
 ('tambien', 27056),
 ('interesar', 26965),
 ('live', 26858),
 ('dar', 26746),
 ('tiempo', 26365),
 ('au

In [41]:
docs = ([' '.join(ele) for ele in data_lemmatized] )


In [ ]:
cv=CountVectorizer() 
word_count_vector=cv.fit_transform(docs)
#word_count_vector.toarray()

# Convert bow_matrix into a DataFrame
bow_df = pd.DataFrame(word_count_vector.toarray())

# Map the column names to vocabulary 
bow_df.columns = cv.get_feature_names()

# Print bow_df
print(bow_df)

In [ ]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

In [ ]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

In [ ]:
# count matrix 
count_vector=cv.transform(docs) 
 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [ ]:
feature_names = cv.get_feature_names() 
 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[0] 
 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df = df[(df != 0).all(1)]
df.sort_values(by=["tfidf"],ascending=False).to_csv("outlets_tfidf.csv")

In [ ]:
df

In [ ]:
word = df.index[1]
content["lowercase"] = content['cleanMessage'].apply(lambda x: x.lower())
word_messages = content[content['lowercase'].str.contains(word)]
word_messages = word_messages['lowercase'].value_counts(ascending=False).rename_axis('unique_messages').reset_index(name='counts')
word_messages.to_csv(word + "_messages.csv")
word_messages

### Repeated messages

In [ ]:
content['Total Interactions']=content['Total Interactions'].astype(str).str.replace(',', '').astype(int)
results = content.groupby(['MessageDescr','Link']).aggregate({'MessageDescr': 'count','Likes': 'sum','Comments': 'sum', 'Shares': 'sum', 'Love': 'sum', 'Wow': 'sum', 'Haha': 'sum', 'Sad': 'sum',
       'Angry': 'sum', 'Care': 'sum','Total Interactions':'sum'})
results.rename(columns={"MessageDescr": "Counts"}, inplace=True)

results.to_csv("mostpopularNewsMedia.csv")

In [ ]:
results.sort_values(by='Total Interactions',ascending=False)

In [ ]:
results.reset_index()
plt.xlim(0,400)
plt.ylim(0,450)

plt.xlabel('Comments')
plt.ylabel('Reactions')
plt.scatter(results.Comments, results.Shares,c='black',label='shares')
plt.scatter(results.Comments, results.Haha,c='purple',label='care emoji')
plt.scatter(results.Comments, results.Wow,c='pink',label='Haha emoji')
plt.scatter(results.Comments, results.Love,c='red',label='Love emoji')
plt.scatter(results.Comments, results.Angry,c='blue',label='Angry emoji')
plt.scatter(results.Comments, results.Sad,c='green',label='Sad emoji')

plt.legend()


## Topic Modeling
### Create the Dictionary and Corpus needed for Topic Modeling

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

## Building the Topic Model

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

## View the topics in LDA model

In [ ]:
# Print the Keyword in the 10 topics
#The weights reflect how important a keyword is to that topic.
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

## Compute Model Perplexity and Coherence Score


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

## Visualize the topics-keywords

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

## Building LDA Mallet Model

In [ ]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = 'mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

## Finding the optimal number of topics for LDA

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values


In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[1]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

##  Finding the dominant topic in each sentence

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

## Find the most representative document for each topic

In [ ]:
#We will find the documents a given topic has contributed to the most and infer the topic by reading that document.
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
pd.set_option('display.max_colwidth', 100)
sent_topics_sorteddf_mallet

In [ ]:
sent_topics_sorteddf_mallet.to_csv("cscwDocs/LDA/DominantTopicsFBGroupsStates.csv",encoding="utf-8-sig",index=False)

In [ ]:
## Topic distribution across documents
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics.dropna(inplace=True)
pd.set_option('display.max_colwidth', None)
df_dominant_topics